In [106]:
import numpy as np
import pandas as pd
import scipy.ndimage
from tqdm import tqdm
from joblib import Parallel, delayed
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook"

In [113]:
from src.imgen import ImageGenerator
from src.utils import tqdm_joblib, plot_dist

In [60]:
NUM_CPUS = 12
NUM_IMAGES = 36
NUM_EPOCHS = 100

NUM_QUBITS = 8
NUM_LAYERS = 2

EPOCH_SAMPLE_SIZE = 10**4
BATCH_SAMPLE_SIZE = 10**3

In [61]:
(x_train_raw, y_train), (x_test_raw, y_test) = tf.keras.datasets.mnist.load_data()
x_train_raw[0].shape

(28, 28)

In [62]:
x_train = np.array([scipy.ndimage.zoom(x, 0.58, order=3) for x in x_train_raw])
x_test = np.array([scipy.ndimage.zoom(x, 0.58, order=3) for x in x_test_raw])
x_train[0].shape

(16, 16)

In [109]:
px.imshow(x_train[0], width=500, height=500, color_continuous_scale='sunset')

In [64]:
def load_and_train(x, num_epochs):
    imgen = ImageGenerator(
        NUM_QUBITS, NUM_LAYERS,
        epoch_sample_size=EPOCH_SAMPLE_SIZE, batch_sample_size=BATCH_SAMPLE_SIZE,
        enable_remapping=True
    )
    imgen.load_image(x, show_figure=False)
    imgen.train(imgen.make_dataset(), num_epochs, show_progress=False)
    return imgen.get_output_distribution_history(), imgen.get_real_distribution()

In [69]:
ind_arr = np.random.choice(range(len(x_train)), size=NUM_IMAGES, replace=False)
ind_arr

array([51283, 39106, 15744, 25478, 20486, 49773, 12685, 19893, 40367,
       24408, 44559,  7792, 44858, 26910, 12369, 25210, 49986, 40506,
       57939, 35980, 19295, 45676, 37810, 46569, 37464, 34667, 58492,
       48837, 41156, 33080, 47323, 41138, 15066, 38597, 10653, 56084])

In [70]:
with tqdm_joblib(tqdm(desc="Training models in parallel", total=NUM_IMAGES)) as progress_bar:
    od_hist_arr = Parallel(n_jobs=NUM_CPUS)(delayed(load_and_train)(x_train[i], NUM_EPOCHS) for i in ind_arr)

Training models in parallel: 100%|██████████| 36/36 [05:54<00:00,  9.83s/it]  


In [71]:
ce_arr = [[cross_entropy(od_hist_arr[i][0][j].flatten(), od_hist_arr[i][1].flatten()) for j in range(NUM_EPOCHS)] for i in range(NUM_IMAGES)]

In [46]:
od_hist_arr[0][0][-1].shape

(16, 16)

In [37]:
# https://stackoverflow.com/questions/47377222/what-is-the-problem-with-my-implementation-of-the-cross-entropy-function

def cross_entropy(predictions, targets, epsilon=1e-12):
    """
    Computes cross entropy between targets (encoded as one-hot vectors)
    and predictions. 
    Input: predictions (N, k) ndarray
           targets (N, k) ndarray        
    Returns: scalar
    """
    predictions = np.clip(predictions, epsilon, 1. - epsilon)
    N = predictions.shape[0]
    ce = -np.sum(targets*np.log(predictions+1e-9))/N
    return ce

In [110]:
df = pd.DataFrame(np.array(ce_arr).T)

In [111]:
all_fig = px.line(
    df, labels={'index': 'Step', 'value': 'Cross Entropy'},
    title='Evolution of Cross Entropy'
)
all_fig.update_layout(
    showlegend=False
)

In [112]:
mean_fig = px.line(
    df.mean(axis=1), labels={'index': 'Step', 'value': 'Cross Entropy'},
    title='Evolution of Mean Cross Entropy'
)
mean_fig.update_layout(
    showlegend=False
)

In [116]:
NUM_ROWS = 6
NUM_COLS = 6

In [148]:
output_collage = make_subplots(rows=NUM_ROWS, cols=NUM_COLS)

for r in range(NUM_ROWS):
    for c in range(NUM_COLS):
        output_collage.add_trace(plot_dist(od_hist_arr[r * NUM_COLS + c][0][-1]).data[0], row=r+1, col=c+1)

output_collage.update_layout(
    width=700, height=700,
)

In [150]:
real_collage = make_subplots(rows=NUM_ROWS, cols=NUM_COLS)

for r in range(NUM_ROWS):
    for c in range(NUM_COLS):
        output_collage.add_trace(plot_dist(od_hist_arr[r * NUM_COLS + c][1]).data[0], row=r+1, col=c+1)

output_collage.update_layout(
    width=700, height=700,
)